In [3]:
%%writefile intents.json
{
  "intents": [
    {
      "tag": "enjoyOfTravel",
      "patterns": ["Do you like travelling?","Are you fond of traveling?", "Do you enjoy traveling?","Do you have a strong desire to see and explore the world around you?","Do you get a sense of fulfillment from seeing new sights and experiencing new things?", "Are you a fan of experiencing different cuisines and lifestyles from around the world?", "Do you get excited about the idea of going on a trip to somewhere you've never been before?","Do you relish the opportunity to go on adventures and try new things?"],
      "responses": ["I absolutely love traveling and exploring new places.", "Yes, I find traveling to be an enriching and exciting experience.", "Definitely, traveling allows me to learn about new cultures and meet new people.", "Absolutely, traveling is one of my favorite hobbies.",  "Yes, there is nothing quite like the feeling of discovering a new destination.", "Without a doubt, traveling broadens my horizons and gives me a new perspective on life.", "Yes, I believe that traveling is an essential part of personal growth and development.", "I can't get enough of traveling, there is always something new to discover and learn.", "Yes, traveling allows me to break out of my comfort zone and challenge myself.", "For me, there is nothing quite like the thrill of exploring new destinations and creating unforgettable memories."]
    },
    {
      "tag": "withWhom",
      "patterns": ["Who do you usually travel with?", "Do you usually go on vacation with family or friends?", "With whom do you like to share your travel experiences?", "Who is your favorite travel companion?", "Do you prefer to travel solo or with a group?", "Who do you think is the best person to travel with?", "Do you enjoy traveling with someone or by yourself?", "Who would you love to go on vacation with that you haven't yet?", "Do you like to travel with people who have similar interests as you?", "Do you think it's important to travel with someone you're close to?"],
      "responses": ["I often travel with my family members, such as my parents or siblings.", "I usually go on trips with my close friends.", "Sometimes, I prefer to travel alone to explore new places and challenge myself.", "I travel with my spouse and kids to make it a family vacation.", "I often travel with my colleagues to attend work-related conferences and events.", "I prefer to travel with a tour group to meet new people and make friends.", "I travel with my significant other to create special memories together.", "I usually travel with my classmates or study group to explore new cultures and learn together.", "Sometimes, I travel with my pet to make it a fun and adventurous experience for both of us.", "I travel with my sports team to compete in tournaments and matches in different cities or countries."]
    },
    
    {
      "tag": "WhichPlace",
      "patterns": ["Which places have you traveled to?","What destinations have you explored?",
"What places have you visited so far?","What are some of the places you have been to?",
"What countries have you visited in your travels?",
"What tourist attractions have you seen?","What places have you had the chance to explore?",
"What sights have you seen on your travels?","What landmarks have you visited?","What notable places have you been to?"],

      "responses": ["I have visited many cities and towns in my country.",
"I have visited several countries and experienced a variety of cultures.",
"I have been to beaches, mountains, and even some islands.",
"I have visited historic sites and museums to learn more about the past.",
"I have explored big cities with lots of hustle and bustle.",
"I have visited small villages where life is more simple and traditional.",
"I have traveled to some remote areas and witnessed breathtaking natural scenery.",
"I have visited some theme parks and had lots of fun with family and friends.",
"I have visited some religious sites and learned about different beliefs and practices.",
"I have visited some places known for their delicious food and beverages."]
    },

    {
      "tag": "Hobbi",
      "patterns": ["Do you like exploring local cuisine?",
"Are you interested in visiting historical sites and museums?",
"Do you prefer outdoor activities such as hiking or surfing?",
"Have you tried adventure activities like bungee jumping or skydiving?",
"Do you enjoy shopping for local souvenirs and crafts?",
"Do you like to attend cultural events such as music festivals or art exhibitions?",
"Do you like to meet new people and make friends while traveling?",
"Are you interested in learning about the local language and customs?",
"Do you prefer to relax and unwind at spas or beaches?",
"Do you like to document your travels through photography or journaling?"],
      "responses": ["I like to explore the local culture, try new foods, and meet new people. I also enjoy visiting historical landmarks and museums to learn about the history and traditions of the place.", "When I visit new places, I like to go on walking tours or hikes to see the natural beauty of the area. I also like to try new activities that are unique to the region, like surfing in Hawaii or skiing in the Alps.", "I love to take photographs and capture the essence of the place. I also like to read about the history and culture of the area before I go, so that I can have a deeper understanding of what I'm seeing and experiencing.", "I enjoy immersing myself in the local lifestyle and experiencing things like the daily markets, the public transportation, and the local music scene. It's a great way to get a sense of the place and its people.", "I like to plan my trips in advance, so that I can make the most of my time in the new place. This involves researching the must-see sights and activities, as well as the best places to eat and drink.", "I enjoy trying new things when I visit new places. This could be as simple as trying a new type of food, or as adventurous as skydiving or bungee jumping.", "I like to take a slow approach when I visit new places, and really soak up the atmosphere of the place. This could involve spending time in local parks, visiting coffee shops, or just people-watching.", "When I visit new places, I like to connect with the local people and learn about their way of life. This could involve taking a cooking class, volunteering, or just striking up a conversation with a stranger.", "I enjoy visiting local art galleries and museums when I visit new places. It's a great way to learn about the local culture and history, and to see some amazing works of art.", "I like to challenge myself physically when I visit new places. This could involve going on a long hike, trying a new sport, or just walking around the city to see the sights."]
    }
  ]
}

Writing intents.json


In [36]:
import nltk
import numpy as np
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader 
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()


def tokenize(sentence):
    """
    split sentence into array of words/tokens
    a token can be a word or punctuation character, or number
    """
    return nltk.word_tokenize(sentence)


def stem(word):
    """
    stemming = find the root form of the word
    examples:
    words = ["organize", "organizes", "organizing"]
    words = [stem(w) for w in words]
    -> ["organ", "organ", "organ"]
    """
    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [34]:
import torch
import torch.nn as nn


class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [38]:
import numpy as np
import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

with open('intents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

38 patterns
4 tags: ['Hobbi', 'WhichPlace', 'enjoyOfTravel', 'withWhom']
138 unique stemmed words: ["'re", "'s", "'ve", 'a', 'about', 'activ', 'adventur', 'and', 'are', 'around', 'art', 'as', 'at', 'attend', 'attract', 'beach', 'been', 'befor', 'best', 'bunge', 'by', 'chanc', 'close', 'companion', 'countri', 'craft', 'cuisin', 'cultur', 'custom', 'desir', 'destin', 'differ', 'do', 'document', 'enjoy', 'event', 'excit', 'exhibit', 'experi', 'experienc', 'explor', 'famili', 'fan', 'far', 'favorit', 'festiv', 'fond', 'for', 'friend', 'from', 'fulfil', 'get', 'go', 'group', 'had', 'have', 'hike', 'histor', 'idea', 'import', 'in', 'interest', 'is', 'it', 'journal', 'jump', 'landmark', 'languag', 'learn', 'lifestyl', 'like', 'local', 'love', 'make', 'meet', 'museum', 'music', "n't", 'never', 'new', 'notabl', 'of', 'on', 'opportun', 'or', 'outdoor', 'peopl', 'person', 'photographi', 'place', 'prefer', 'relax', 'relish', 'see', 'seen', 'sens', 'share', 'shop', 'sight', 'similar', 'site', 'skyd

In [39]:
print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

final loss: 0.0001
training complete. file saved to data.pth


In [42]:
import random
import json

import torch


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Turism based bot"

def get_response(msg):
    sentence = tokenize(msg)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                return random.choice(intent['responses'])
    
    return "I do not understand..."


if __name__ == "__main__":
    print("Let's chat! (type 'quit' to exit)")
    while True:
        # sentence = "do you use credit cards?"
        sentence = input("You: ")
        if sentence == "quit":
            break

        resp = get_response(sentence)
        print(resp)


Let's chat! (type 'quit' to exit)
You: which place do you want to go to travel
I have visited small villages where life is more simple and traditional.
You: Are you interested in visiting historical sites and museums?
I like to explore the local culture, try new foods, and meet new people. I also enjoy visiting historical landmarks and museums to learn about the history and traditions of the place.
You: What countries have you visited in your travels?
I have visited several countries and experienced a variety of cultures.
You: Who do you usually travel with?
I usually travel with my classmates or study group to explore new cultures and learn together.


KeyboardInterrupt: ignored